## **Importing Libraries**

In [2]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
from os import listdir
import cv2
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import imutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image  as mpimg
import tensorflow as tf
from keras.preprocessing import image
import datetime

### DataSet

In [4]:
print("The number of images with facemask labelled 'yes':",len(os.listdir('/data/train/with_mask')))
print("The number of images with facemask labelled 'no':",len(os.listdir('/data/train/with_mask')))

The number of images with facemask labelled 'yes': 690
The number of images with facemask labelled 'no': 686


In [5]:
def data_summary(main_path):
    
    yes_path = main_path+'with_mask'
    no_path = main_path+'without_mask'
        
    # number of files (images) that are in the the folder named 'yes' that represent tumorous (positive) examples
    m_pos = len(listdir(yes_path))
    # number of files (images) that are in the the folder named 'no' that represent non-tumorous (negative) examples
    m_neg = len(listdir(no_path))
    # number of all examples
    m = (m_pos+m_neg)
    
    pos_prec = (m_pos* 100.0)/ m
    neg_prec = (m_neg* 100.0)/ m
    
    print(f"Number of examples: {m}")
    print(f"Percentage of positive examples: {pos_prec}%, number of pos examples: {m_pos}") 
    print(f"Percentage of negative examples: {neg_prec}%, number of neg examples: {m_neg}") 
    
augmented_data_path = '//data/train/'    
data_summary(augmented_data_path)

Number of examples: 1508
Percentage of positive examples: 50.06631299734748%, number of pos examples: 755
Percentage of negative examples: 49.93368700265252%, number of neg examples: 753


## **Data** **Spliting**
80% training data
20% testing data 


In [8]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    dataset = []
    
    for unitData in os.listdir(SOURCE):
        data = SOURCE + unitData
        if(os.path.getsize(data) > 0):
            dataset.append(unitData)
        else:
            print('Skipped ' + unitData)
            print('Invalid file i.e zero size')
    
    train_set_length = int(len(dataset) * SPLIT_SIZE)
    test_set_length = int(len(dataset) - train_set_length)
    shuffled_set = random.sample(dataset, len(dataset))
    train_set = dataset[0:train_set_length]
    test_set = dataset[-test_set_length:]
       
    for unitData in train_set:
        temp_train_set = SOURCE + unitData
        final_train_set = TRAINING + unitData
        copyfile(temp_train_set, final_train_set)
    
    for unitData in test_set:
        temp_test_set = SOURCE + unitData
        final_test_set = TESTING + unitData
        copyfile(temp_test_set, final_test_set)
        
        
YES_SOURCE_DIR = "/data/with_mask/"
TRAINING_YES_DIR = "/data/train/with_mask/"
TESTING_YES_DIR = "/data/test/with_mask/"
NO_SOURCE_DIR =  "/data/without_mask/"
TRAINING_NO_DIR = "/data/train/without_mask/"
TESTING_NO_DIR =  "/data/test/without_mask/"
split_size = .8
split_data(YES_SOURCE_DIR, TRAINING_YES_DIR, TESTING_YES_DIR, split_size)
split_data(NO_SOURCE_DIR, TRAINING_NO_DIR, TESTING_NO_DIR, split_size)

Describtion

In [ ]:
print("The number of images with facemask in the training set labelled 'yes':", len(os.listdir('/data/train/with_mask')))
print("The number of images with facemask in the test set labelled 'yes':", len(os.listdir('/data/test/with_mask/')))
print("The number of images without facemask in the training set labelled 'no':", len(os.listdir('/data/train/without_mask/')))
print("The number of images without facemask in the test set labelled 'no':", len(os.listdir('/data/test/without_mask/')))

### CNN Model

In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(100, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [11]:
TRAINING_DIR ="/data/train/"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "/data/test/"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

Found 1508 images belonging to 2 classes.
Found 470 images belonging to 2 classes.


In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 100)     2800      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 100)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 100)       90100     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 100)       0         
_________________________________________________________________
flatten (Flatten)            (None, 129600)            0         
_________________________________________________________________
dropout (Dropout)            (None, 129600)            0         
_________________________________________________________________
dense (Dense)                (None, 50)                6

## Training

In [13]:
history = model.fit_generator(train_generator,
                              epochs=30,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/30
151/151 [==============================] - 269s 2s/step - loss: 0.7237 - acc: 0.5794 - val_loss: 0.4607 - val_acc: 0.8936
INFO:tensorflow:Assets written to: model-001.model/assets
Epoch 2/30
151/151 [==============================] - 144s 956ms/step - loss: 0.4969 - acc: 0.8493 - val_loss: 0.3727 - val_acc: 0.9362
INFO:tensorflow:Assets written to: model-002.model/assets
Epoch 3/30
151/151 [==============================] - 145s 962ms/step - loss: 0.4311 - acc: 0.8892 - val_loss: 0.3379 - val_acc: 0.9489
INFO:tensorflow:Assets written to: model-003.model/assets
Epoch 4/30
151/151 [==============================] - 144s 954ms/step - loss: 0.4332 - acc: 0.8688 - val_loss: 0.3871 - val_acc: 0.8915
Epoch 5/30
151/151 [==============================] - 144s 951ms/step - loss: 0.3814 - acc: 0.8978 - val_loss: 0.3159 - val_acc: 0.9404
INFO:tensorflow:Assets written to: model-005.model/assets
Epoch 6/30
151/151 [==============================] - 145s 960ms/step - loss: 0.3754 - acc:

## Save

In [17]:
model.save('model.h5')

## load

In [37]:
model=tf.keras.models.load_model("mymodel.h5")

for Live camera 0 in VideoCapture
## OR
Path in String "video.mp4"

In [ ]:
labels_dict={0:'without_mask',1:'with_mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

size = 4
cap = cv2.VideoCapture(0) #Use camera 0 

# We load the xml file
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

while cap.isOpened():
    _,img=cap.read()
    face=face_cascade.detectMultiScale(img,scaleFactor=1.1,minNeighbors=4)
    for(x,y,w,h) in face:
        face_img = img[y:y+h, x:x+w]
        cv2.imwrite('temp.jpg',face_img)
        test_image=image.load_img('temp.jpg',target_size=(150,150,3))
        test_image=image.img_to_array(test_image)
        test_image=np.expand_dims(test_image,axis=0)
        pred=mymodel.predict_classes(test_image)[0][0]
        if pred==1:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,0,255),3)
            cv2.putText(img,'NO MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        else:
            cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            cv2.putText(img,'MASK',((x+w)//2,y+h+20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),3)
        datet=str(datetime.datetime.now())
        cv2.putText(img,datet,(400,450),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),1)
    #cv2.imwrite("mask_save.mp4",img)      
    cv2.imshow('img',img)
    
    if cv2.waitKey(1)==ord('q'):
      break
    
cap.release()
cv2.destroyAllWindows()